In [1]:
from tensorflow.python.client import device_lib
    device_lib.list_local_devices()

2022-07-13 16:23:25.411668: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-13 16:23:27.083341: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-13 16:23:27.278917: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-13 16:23:27.288406: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18413635136082971040
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14256898048
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5316161492309185616
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [1]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

from imutils import face_utils
import dlib
import pandas as pd
import json


import pickle

%matplotlib inline


In [2]:
def rotate_image(image, landmarks):
    left_eye_x=left_eye_y=right_eye_x=right_eye_y=0
    for (x,y) in landmarks[36:41]:
        left_eye_x+=x
        left_eye_y+=y
    for (x,y) in landmarks[42:47]:
        right_eye_x+=x
        right_eye_y+=y
    left_eye_x/=6
    left_eye_y/=6
    right_eye_x/=6
    right_eye_y/=6
    theta=math.degrees(math.atan((right_eye_y-left_eye_y)/(right_eye_x-left_eye_x)))
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, theta, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    #print(left_eye_x, left_eye_y, right_eye_x, right_eye_y,theta,img.shape)
    return result

### save path here 

In [7]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'

datFile = training + "shape_predictor_68_face_landmarks.dat"
#scaled_anger = scaled + 'scaled_disgust/one/''
print(training)
print(validation)
print(scaled)



/media/windows/training/
/media/windows/validation/
/media/windows/scaled/


### create list myfiles with the filename (start with surprise)

In [8]:
os.chdir(training+"train_happiness_01/")
path = os.getcwd()
#print(path)
myfiles = os.listdir()
#print(myfiles[0])
print(len(myfiles))

16072


### create json_sorted with only last 26 numbers 

In [9]:
with open(training+"label_happiness/meta_happiness.json", 'r',encoding = "utf-8")as f:
    original_data = json.load(f) #original json file
    
    #print((original_data[1]))

#type(json_data)  list 
json_sorted = sorted(original_data, key=lambda img: (img['filename']))

for i in range(len(json_sorted)):
    json_sorted[i]["filename"] = json_sorted[i]["filename"][-26:]

#print(json_sorted[0]["filename"])



    
        

### made_metadata_anger is list made of dictionaries

In [6]:
made_metadata_happiness = [] #becomes new json file with filename and cut 
#only ones that matches with the json file is uploaded in new json file 
k=0
for filename in myfiles:
    dic = {}
    #print("filename is :", filename)
    for i in range(len(json_sorted)):
        #print(json_sorted[i]["filename"])
        if filename == json_sorted[i]["filename"]:
            dic['filename'] = filename
            dic['cut'] = json_sorted[i]["annot_A"]["boxes"]
            made_metadata_happiness.append(dic)
            break
    k+=1
    
    if k == 1000:
        k = 0
        print("thou")

print(len(made_metadata_happiness))

thou
thou
thou
thou
thou
thou
thou
thou
thou
thou
thou
thou
thou
thou
thou
thou
15849


In [10]:
with open(training + "label_happiness/meta_happiness_01.json", 'w', encoding = 'utf-8') as make_file:
    json.dump(made_metadata_happiness, make_file, indent = "\t")


In [11]:
def create_landmark(image):
    ALL = list(range(0, 68))
    RIGHT_EYEBROW = list(range(17, 22))
    LEFT_EYEBROW = list(range(22, 27))
    RIGHT_EYE = list(range(36, 42))
    LEFT_EYE = list(range(42, 48))
    NOSE = list(range(27, 36))
    MOUTH_OUTLINE = list(range(48, 61))
    MOUTH_INNER = list(range(61, 68))
    JAWLINE = list(range(0, 17))

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(datFile)

    face_detector = detector(image,1) #detecting face from the image 
    try:
        landmarks = predictor(image, face_detector[0]) 
    except:
        return 0
    landmarks_list = []

    for p in landmarks.parts():
        landmarks_list.append([p.x, p.y])
    
    return landmarks_list



In [20]:
with open(training+"label_happiness/meta_happiness_01.json", 'r')as f:
    json_data = json.load(f)
    
    print(len(json_data))

counter = 0
thou_counter = 0
saved = 0
#type(json_data)  list 
for i in range(len(json_data)): #len(json_data)
    counter+=1
    if counter == 1000:
        counter = 0
        thou_counter+=1
        print(thou_counter)
        print(saved)
    img_name = json_data[i]["filename"]
    img_cut = json_data[i]["cut"]  
    img = training + "train_happiness_01/"+img_name
    #print(img_name)
    #print(img_cut)
    input_image = cv2.imread(img) 
    #input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    
    maxX = img_cut["maxX"] 
    maxY = img_cut["maxY"] 
    minX = img_cut["minX"] 
    minY = img_cut["minY"]
    if maxX<0 or maxY <0 or minX<0 or minY < 0:
        continue
    dst = input_image[int(minY):int(maxY), int(minX):int(maxX)].copy()
    resized_image = cv2.resize(dst, (224,224))
    #plt.imshow(resized_image)
    #plt.show()

    landmark_list = create_landmark(resized_image)
    if landmark_list == 0: 
        continue
    rotated_image = rotate_image(resized_image, landmark_list)
    #plt.imshow(rotated_image)
    #plt.show()
    #print(landmark_list)
    #resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    cv2.imwrite(scaled + "scaled_happiness/one/"+img_name, rotated_image)
    saved=+1

    
 

15849
1
1
2
1
3
1


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG


4
1


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS

5
1
6
1
7
1
8
1


...

10
1
11
1
12
1


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS

13
1
14
1


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS

15
1
